In [1]:
import taxon_keys

INFO:Note: detected 144 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:Note: NumExpr detected 144 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:NumExpr defaulting to 8 threads.


In [2]:
non_moth_species_path = '/bask/homes/f/fspo1218/amber/projects/on_device_classifier/01_data_download/species_lists/moth_trap_non-moths_21-08-2023.csv'
moth_species_path = '/bask/homes/f/fspo1218/amber/projects/on_device_classifier/01_data_download/species_lists/uksi-macro-moths.csv'

non_moth_output_path = '/bask/homes/f/fspo1218/amber/projects/gbif_download_standalone/output_data/keys/non-moth_data.csv'
moth_output_path = '/bask/homes/f/fspo1218/amber/projects/gbif_download_standalone/output_data/keys/moth_data.csv'

column_name = 'taxon'

In [3]:
import pandas as pd

moth_df = pd.read_csv(moth_species_path)

moth_df.head()

,family_taxon,taxon,preferred_authority,common_name,name_tvk,rtvk,organism_key
0,Cossidae,Cossus cossus,"(Linnaeus, 1758)",Goat Moth,NBNSYS0000005662,NBNSYS0000005662,NBNORG0000009013
1,Cossidae,Phragmataecia castaneae,"(Hübner, 1790)",Reed Leopard,NHMSYS0000503737,NHMSYS0000503737,NBNORG0000058754
2,Cossidae,Zeuzera pyrina,"(Linnaeus, 1761)",Leopard Moth,NBNSYS0000005661,NBNSYS0000005661,NBNORG0000009012
3,Drepanidae,Achlya flavicornis,"(Linnaeus, 1758)",Yellow Horned,NBNSYS0000005732,NBNSYS0000005732,NBNORG0000009060
4,Drepanidae,Cilix glaucata,"(Scopoli, 1763)",Chinese Character,NBNSYS0000005723,NBNSYS0000005723,NBNORG0000009054


In [4]:
non_moth_df = pd.read_csv(non_moth_species_path)
non_moth_df.rename(columns={ non_moth_df.columns[0]: "taxon" }, inplace = True)

non_moth_df.head()

,taxon,Common name,Group,Kingdom,Order,Family,No. of records,First record,Last record
0,Limnephilus lunatus,NaN,insect - caddis fly (Trichoptera),Animalia,Trichoptera,Limnephilidae,440,19/06/2020,19/08/2023
1,Melolontha melolontha,Common Cockchafer,insect - beetle (Coleoptera),Animalia,Coleoptera,Scarabaeidae,394,13/05/2019,09/08/2023
2,Limnephilus marmoratus,NaN,insect - caddis fly (Trichoptera),Animalia,Trichoptera,Limnephilidae,331,17/07/2013,20/08/2023
3,Nicrophorus humator,Black Sexton Beetle,insect - beetle (Coleoptera),Animalia,Coleoptera,Silphidae,303,17/07/2020,16/08/2023
4,Glyphotaelius pellucidus,NaN,insect - caddis fly (Trichoptera),Animalia,Trichoptera,Limnephilidae,291,06/06/2020,17/08/2023


In [5]:
def process_species_list(data, column_name, output_path):
    # fetch species names from the list
    #data = pd.read_csv(file_path, index_col=False)
    species_list = []
    for indx in data.index:
        species_list.append(data[column_name][indx])

    data_final = pd.DataFrame(
        columns=[
            "taxon_key_gbif_id",
            "order_name",
            "family_name",
            "genus_name",
            "search_species_name",
            "gbif_species_name",
            "confidence",
            "status",
            "match_type",
        ],
        dtype=object,
    )

    # fetch taxonomy data from GBIF
    for name in species_list:
        data = taxon_keys.get_gbif_key_backbone(name)
        data_final = pd.concat([data_final, data], ignore_index=True)

    # save the file
    data_final.to_csv(output_path, index=False)

In [6]:
# moth
#process_species_list(moth_df, 'taxon', moth_output_path)

In [7]:
# non moth 
#process_species_list(non_moth_df, 'taxon', non_moth_output_path)

In [8]:
non_moth_output_path

'/bask/homes/f/fspo1218/amber/projects/gbif_download_standalone/output_data/keys/non-moth_data.csv'

In [9]:
import fetch_gbif

In [10]:
write_directory = '/bask/homes/f/fspo1218/amber/data/binary_moth_training/nonmoth/'
species_key_filepath = non_moth_output_path
max_images_per_species = 500
resume_session= True

LIMIT_DOWN = 300  # GBIF API parameter for max results per page
MAX_SEARCHES = 11000  # maximum no. of points to iterate
moth_data = pd.read_csv(species_key_filepath)

moth_data.head()

limit_down=300
max_searches = 11000

In [11]:
moth_data.head()

,taxon_key_gbif_id,order_name,family_name,genus_name,search_species_name,gbif_species_name,confidence,status,match_type
0,1442474,Trichoptera,Limnephilidae,Limnephilus,Limnephilus lunatus,Limnephilus lunatus,99,ACCEPTED,EXACT
1,4990995,Coleoptera,Scarabaeidae,Melolontha,Melolontha melolontha,Melolontha melolontha,99,ACCEPTED,EXACT
2,1442615,Trichoptera,Limnephilidae,Limnephilus,Limnephilus marmoratus,Limnephilus marmoratus,99,ACCEPTED,EXACT
3,1039003,Coleoptera,Silphidae,Nicrophorus,Nicrophorus humator,Nicrophorus humator,99,ACCEPTED,EXACT
4,1442831,Trichoptera,Limnephilidae,Glyphotaelius,Glyphotaelius pellucidus,Glyphotaelius pellucidus,99,ACCEPTED,EXACT


In [ ]:
import pandas as pd

MAX_DATA_SP = 500

fetch_gbif.get_gbif_images(moth_data, True, '/bask/homes/f/fspo1218/amber/data/binary_moth_training/nonmoth/', max_images_per_species, limit_down, max_searches)

Already downloaded for Limnephilus lunatus
Already downloaded for Melolontha melolontha
Already downloaded for Limnephilus marmoratus
Already downloaded for Nicrophorus humator
Already downloaded for Glyphotaelius pellucidus
Already downloaded for Mystacides longicornis
Already downloaded for Vespula vulgaris
Already downloaded for Harmonia axyridis
Already downloaded for Nicrophorus investigator
Already downloaded for Limnephilus affinis
Already downloaded for Oecetis ochracea
Already downloaded for Pentatoma rufipes
Already downloaded for Tipula paludosa
Already downloaded for Vespa crabro
Already downloaded for Limnephilus auricula
Already downloaded for Stenophylax permistus
Already downloaded for Trichocera annulata
Already downloaded for Limnephilus rhombicus
Already downloaded for Limnephilus sparsus
Already downloaded for Amphimallon solstitiale
Already downloaded for Elasmostethus interstinctus
Already downloaded for Necrodes littoralis
Already downloaded for Chrysoperla carne